# Mediods Analysis

This experiment has the purpose of analysing the medoid of some of the clusters defined in the previous experiment. According to it, the best number of cluster is 82 regarding the square of the 2-norm distance, and 802 regarding the silhouette score. With 802 clusters, the silhouette score was 0.1, a considerably low value. Therefore, we won't consider it in the next experiments, implying that 82 is the best number of clusters.

In [1]:
#!pip install numpy==1.15 wordcloud

In [2]:
import sys
import os
sys.path.append('../')
from src import reader as r
from src import visualization as v

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
%matplotlib notebook
import numpy as np
import pandas as pd
import sklearn
print(sklearn.__version__)
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics import pairwise_distances

0.20.0


In [4]:
X = r.readBOW()
npX = X
print(X[:10])
print(X.shape)

['word2vec.csv', 'bags.csv', 'health.txt', 'health-dataset.zip', 'health-dataset']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(13229, 1203)


In [5]:
from importlib import reload
reload(r)
news = r.readNews()
print(news.head(10))
print(news.shape)

['word2vec.csv', 'bags.csv', 'health.txt', 'health-dataset.zip', 'health-dataset']
Read 13229 lines
                   id                    publish_date  \
0  576880531301801984  Sat Mar 14 23:00:11 +0000 2015   
1  576820122666471424  Sat Mar 14 19:00:08 +0000 2015   
2  576744652717461504  Sat Mar 14 14:00:15 +0000 2015   
3  576736754436304896  Sat Mar 14 13:28:52 +0000 2015   
4  576736614766010368  Sat Mar 14 13:28:18 +0000 2015   
5  576548368740052992  Sat Mar 14 01:00:17 +0000 2015   
6  576518190286536704  Fri Mar 13 23:00:22 +0000 2015   
7  576494177971732480  Fri Mar 13 21:24:57 +0000 2015   
8  576472874946433024  Fri Mar 13 20:00:18 +0000 2015   
9  576464606551490560  Fri Mar 13 19:27:27 +0000 2015   

                                       headline_test  
0  An abundance of online info can turn us into e...  
1  A plant-based diet that incorporates fish may ...  
2  It doesn't take much to damage your hearing at...  
3  RT @CNN: Forever young? Discover this island’s...

In [6]:
from MulticoreTSNE import MulticoreTSNE as TSNE

smp_sz = X.size
tsne_bow = TSNE(n_components=2, perplexity=10, verbose=True, n_jobs=-1)#500
tsne_bow_result = tsne_bow.fit_transform(X)

## Computing the Medoids
In here, we compute the medoids as the closest point to the centroids of the clusters.

In [7]:
best_K = 71
print("#############################")
print("Best K =", best_K)
print("Applying K-means")
best_cluster = KMeans(n_clusters=best_K, n_jobs=-1)
best_cluster_result = best_cluster.fit(X)
print("Finished")
print("#############################")    

#############################
Best K = 71
Applying K-means
Finished
#############################


In [8]:
y_pred = best_cluster_result.labels_
arrays=[]
for i in range(best_K):
    dist = np.linalg.norm(best_cluster_result.cluster_centers_[i]-npX[y_pred==i],axis=1)
    index = np.argmin(dist)
    arrays.append(npX[y_pred==i][index])   

medoids = np.vstack(arrays)
print(medoids.shape)
print(medoids)

(71, 1203)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [9]:
l = []
for i in range(best_K):
    l.append(np.where(np.all(npX==medoids[i],axis=1))[0][0]) # these last [0][0] return the index, since the output of this command is (array([id]),)
    
medoids_index = np.array(l)
print(medoids_index)
print(len(np.unique(medoids_index)))

[ 2464  2971  2923  2278  9819  2206  4107   901    99  3257  8126   331
  2190   777 10133  3697  2944   366  1168  5067 12783  2212  4614  7429
  9420   569  3357 11001 11178 10224 12243  3467  3872 10681  6852 10782
  7093  5869  6519  4723  3093  3043  3691  4566  2293  9979  4897  7576
  2751  3019  6089  2785   805  2938 11845  4130  2725  1051 12250 12311
 11151 10713  9451  2798  5033  9356  8016  5435  7765 12965  4387]
71


## Selecting medoids neighbors
Here the medoids neighbors are selected in order to plot them and see if the clusters make sense.

In [10]:
nm = 5 #number of medoids to be analyzed
nn = 5 #number of neighbor of each medoids
np.random.seed(42)
selected_medoids_index = np.random.randint(0,best_K,nm)
selected_medoids = medoids[selected_medoids_index]
print("Random indexes:",selected_medoids_index)
indexes_per_medoid = []

for i in range(nm):
    p = selected_medoids[i]
    aux = npX[y_pred==selected_medoids_index[i]]
    d = np.linalg.norm(aux-p,axis=1)
    points = []
    points.append(medoids_index[selected_medoids_index[i]])
    
    for j in range(nn):
        min_d = d.argmin()
        while (d[min_d] == 0.):
            d[min_d] = np.inf
            min_d = d.argmin()
        points = points + (np.where(np.all(npX==aux[min_d],axis=1))[0]).tolist()
        d[min_d] = np.inf
        
    indexes_per_medoid.append(np.unique(points).tolist())

    
print("Points:",indexes_per_medoid)
print(len(indexes_per_medoid))

Random indexes: [51 14 60 20 23]
Points: [[681, 1121, 2074, 2692, 2785, 2983], [9512, 9661, 10133, 11765, 12126, 12233], [9301, 10306, 10331, 11151, 11264, 11579], [6225, 7116, 7305, 7975, 11086, 12109, 12783], [3392, 4340, 7050, 7429, 7745, 11576]]
5


## Analyzing the Medoids
Here we start the analysis of the medoids and their neighbors.

### Exporting the news of the medoids and clusters

In [11]:
import os 

os.makedirs('../output/clusters/', exist_ok=True)
os.makedirs('../output/medoids/', exist_ok=True)


print("Random indexes:",selected_medoids_index)
for i in range(best_K):
    np.savetxt("../output/clusters/k_"+str(i)+".txt", news[y_pred==i].values, fmt='%s', encoding='utf8')
for i in range(selected_medoids_index.size):
    p = indexes_per_medoid[i]
    np.savetxt("../output/medoids/k_"+str(selected_medoids_index[i])+".txt", np.take(news,p,0).values, fmt='%s')

Random indexes: [51 14 60 20 23]


### News of the medoids and their neighbors

In [12]:
for p in indexes_per_medoid:
    info = np.take(news,p,0).loc[:,"headline_test"].values
    print("----------------------------")
    for i in range(info.size):
        print(info[i],"\n")
print("----------------------------")

----------------------------
After a miserable 30th birthday, Sarah Evans lost 120 pounds! 

You might not recognize former @tigers pitcher Denny McLain. He's lost 162 pounds! 

Kia Horton, 38, went from 319 pounds to 142 pounds. See how she did it! 

Woman shakes off 170 pounds 

Truck driver dances off 97 pounds 

Best. Video. Ever. Truck driver dances off 97 pounds 

----------------------------
VIDEO: Improving palliative care in Mexico 

VIDEO: 'Full investigation' when care fails 

'Take care complaints seriously' 

Elderly care: 'Who will care for me?' 

Privatisation 'hurting health care' 

VIDEO: Psychological scars of intensive care 

----------------------------
GP workload harming care - BMA poll 

Watchdog ranks GP surgeries by risk 

Patients 'struggling to book with GP' 

'Plight' of some rural GP surgeries 

Seeing GP 'getting more difficult' 

VIDEO: Charging to see a GP? 

----------------------------
Put the Bottle Down: Health Risks of #Drinking 

A #newstudy uncove

### Generating cloud of words

In [13]:
%matplotlib inline
from wordcloud import WordCloud
import matplotlib
import matplotlib.pyplot as plt

os.makedirs('../output/wordcloud_clusters/', exist_ok=True)
os.makedirs('../output/wordcloud_medoids/', exist_ok=True)

# clusters
for i in range(best_K):
    info = ' '.join(news[y_pred==i].loc[:,"headline_test"])
    wordcloud = WordCloud(random_state=42).generate(info)
    fig = plt.figure()
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")    
    fig.savefig("../output/wordcloud_clusters/k_"+str(i))
    plt.close(fig)

#medoids
for i in range(selected_medoids_index.size):
    p = indexes_per_medoid[i]
    info = ' '.join(np.take(news,p,0).loc[:,"headline_test"])
    wordcloud = WordCloud(random_state=42).generate(info)
    fig = plt.figure()
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")    
    fig.savefig("../output/wordcloud_medoids/k_"+str(selected_medoids_index[i]))
    plt.close(fig)
